# 프로젝트 추적, 로깅 시작

In [2]:
import wandb

wandb.init()

# 실행 이름 설정
project_name = 'First wandb'
wandb.run.name = project_name
wandb.run.save()

True

## hyperparameters wandb로 전달

In [3]:
learning_rate = 0.01
epochs = 100
batch_size = 10000

args = {
    "learning_rate" : learning_rate,
    "epochs" : epochs,
    "batch_size" : batch_size
}

wandb.config.update(args)

## 로그 전달

In [4]:
wandb.log({"Training loss" : 10})

# 실제 사용예시

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import wandb

wandb.init(project="sunho_test")
wandb.run.name = "test"
wandb.run.save()

batch_size = 4
learning_rate = 0.001
epochs = 5

args = {
    "learning_rate" : learning_rate,
    "epochs" : epochs,
    "batch_size" : batch_size
}

wandb.config.update(args)

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

# 2. Define a CNN
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:', device)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
 
net = Net().to(device)
 
# 3. Define a Loss Function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
 
# 4. Train the network
for epoch in range(epochs):  # loop over the dataset multiple times
 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
 
        # zero the parameter gradients
        optimizer.zero_grad()
 
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            wandb.log({"Training loss": running_loss / 2000})
            running_loss = 0.0
 
# 모델 weight 저장
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)
 
print('Finished Training')


100%|██████████| 170498071/170498071 [13:04<00:00, 217195.91it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
device: cpu
[1,  2000] loss: 2.225
[1,  4000] loss: 1.919
[1,  6000] loss: 1.698
[1,  8000] loss: 1.617
[1, 10000] loss: 1.548
[1, 12000] loss: 1.480
[2,  2000] loss: 1.421
